# LangGraph Artifacts Catalog

This notebook catalogs all proposed **nodes** and **tools** for the OSC LangGraph architecture.

Each artifact follows the same pattern as `velocity_humanizer.ipynb`:
1. **Purpose** - What it does
2. **Input/Output Contract** - Data shapes
3. **Tool Function Signature** - Pure function interface
4. **LangGraph Node Wrapper** - Async state handler
5. **OpenAI Function Schema** - For LLM tool calling (optional)
6. **Example Usage**

---

## Artifact Categories

| Category | Artifacts |
|----------|----------|
| **Generation** | `generate_clip_from_nl`, `generate_composition_from_nl` |
| **Validation** | `validate_sml_clip`, `validate_sml_composition` |
| **Expression** | `humanize_velocities`, `apply_cc_template`, `apply_pitch_bend` |
| **Transformation** | `transpose_clip`, `quantize_clip`, `apply_swing`, `invert_clip`, `retrograde_clip` |
| **Harmony** | `generate_chord_track`, `generate_bassline`, `arpeggiate_chords` |
| **Playback** | `preview_clip`, `preview_composition` |
| **Storage** | `store_clip`, `search_clips`, `load_dsl_project` |
| **Export** | `export_clip_to_midi`, `export_composition_to_midi` |
| **QA** | `lint_clip`, `summarize_clip` |

---
# 1. GENERATION TOOLS
---

## 1.1 generate_clip_from_nl

**Purpose**: Convert natural language prompt to SML clip using LangGraph + OpenAI.

**Wraps**: `OSCFacade.natural_language_clip_to_sml` / `clip_graph.generate_sml_clip`

**Input**: Natural language prompt string

**Output**: SML clip dict with bars/items structure

In [ ]:
from typing import Any, Dict, Optional, TypedDict

class GenerateClipParams(TypedDict, total=False):
    prompt: str                      # Natural language description
    num_bars: int                    # Target number of bars (default: 2)
    key: str                         # Key signature (e.g., "C", "F#")
    mode: str                        # "major" or "minor"
    meter: str                       # Time signature (e.g., "4/4", "3/4")
    style: str                       # Style hint (e.g., "jazz", "classical", "lofi")
    instrument: str                  # Target instrument (e.g., "piano", "bass", "strings")
    tempo_bpm: int                   # Reference tempo for duration choices


def generate_clip_from_nl(
    params: GenerateClipParams,
) -> Dict[str, Any]:
    """Generate an SML clip from natural language.
    
    Contract:
    - Input: GenerateClipParams with at minimum a `prompt`
    - Output: SML clip dict with structure:
        {
            "name": str,
            "bars": [
                {
                    "bar_index": int,
                    "items": [{"note": "C4", "duration": "quarter"}, ...]
                }
            ]
        }
    - Raises: ValueError if generation fails
    
    This wraps the existing clip_graph.generate_sml_clip flow.
    """
    # Implementation wraps OSCFacade.natural_language_clip_to_sml
    pass

In [ ]:
# LangGraph Node Wrapper

class ClipGenerationState(TypedDict, total=False):
    prompt: str
    generate_params: GenerateClipParams
    sml_clip: Dict[str, Any]
    error: str


async def generate_clip_node(state: ClipGenerationState) -> ClipGenerationState:
    """LangGraph node: Generate SML clip from prompt."""
    if state.get("error"):
        return state
    
    prompt = state.get("prompt")
    if not prompt:
        return {**state, "error": "No prompt provided"}
    
    params = state.get("generate_params", {})
    params["prompt"] = prompt
    
    try:
        sml_clip = generate_clip_from_nl(params)
        return {**state, "sml_clip": sml_clip}
    except Exception as e:
        return {**state, "error": f"Generation failed: {e}"}

---
# 2. VALIDATION TOOLS
---

## 2.1 validate_sml_clip

**Purpose**: Validate and normalize an SML clip, reporting errors/warnings.

**Wraps**: `sml_ast.clip_from_smil_dict` with error capture

**Checks**:
- Bar durations fill correctly (no overfull/underfull bars)
- Pitches are valid MIDI range (0-127)
- Velocities are valid (1-127)
- Duration tokens are recognized
- CC values in range (0-127)
- Pitch bend values in range (0-16383)

In [ ]:
from typing import List, Literal

class ValidationIssue(TypedDict):
    level: Literal["error", "warning", "info"]
    bar_index: Optional[int]
    item_index: Optional[int]
    message: str
    auto_fixed: bool


class ValidationResult(TypedDict):
    valid: bool
    issues: List[ValidationIssue]
    normalized_clip: Optional[Dict[str, Any]]  # Auto-fixed version if possible


class ValidateClipParams(TypedDict, total=False):
    auto_fix: bool                   # Attempt to fix simple issues (default: True)
    strict: bool                     # Fail on warnings too (default: False)
    meter: str                       # Expected meter for bar duration check
    clamp_velocities: bool           # Clamp velocities to 1-127 (default: True)
    clamp_cc: bool                   # Clamp CC values to 0-127 (default: True)


def validate_sml_clip(
    sml_clip: Dict[str, Any],
    params: Optional[ValidateClipParams] = None,
) -> ValidationResult:
    """Validate an SML clip and optionally auto-fix issues.
    
    Contract:
    - Input: SML clip dict
    - Output: ValidationResult with issues list and optionally normalized clip
    
    Auto-fix behaviors:
    - Clamp velocities to [1, 127]
    - Clamp CC values to [0, 127]
    - Round durations to nearest valid unit
    - Add missing bar_index fields
    """
    pass

In [ ]:
# LangGraph Node Wrapper

class ValidationState(TypedDict, total=False):
    sml_clip: Dict[str, Any]
    validation_params: ValidateClipParams
    validation_result: ValidationResult
    error: str


async def validate_clip_node(state: ValidationState) -> ValidationState:
    """LangGraph node: Validate and normalize SML clip."""
    if state.get("error"):
        return state
    
    sml_clip = state.get("sml_clip")
    if not sml_clip:
        return {**state, "error": "No sml_clip to validate"}
    
    params = state.get("validation_params", {})
    result = validate_sml_clip(sml_clip, params)
    
    # Replace clip with normalized version if available
    if result.get("normalized_clip"):
        state = {**state, "sml_clip": result["normalized_clip"]}
    
    if not result["valid"]:
        errors = [i for i in result["issues"] if i["level"] == "error"]
        return {**state, "validation_result": result, "error": f"Validation failed: {len(errors)} errors"}
    
    return {**state, "validation_result": result}

---
# 3. EXPRESSION TOOLS
---

## 3.1 humanize_velocities

**Purpose**: Apply metrical accents and random jitter to note velocities.

**See**: `velocity_humanizer.ipynb` for full implementation details.

## 3.2 apply_cc_template

**Purpose**: Apply predefined CC curves (mod wheel, expression, filter) to bars.

**Templates**:
- `crescendo` - Gradual increase over bar
- `diminuendo` - Gradual decrease over bar
- `swell` - Rise and fall (pad/strings)
- `tremolo` - LFO-style oscillation (CC1)
- `filter_sweep` - EDM-style filter cutoff (CC74)
- `pedal_sustain` - Piano pedal events (CC64)

In [ ]:
from typing import Literal, Union

class CCTemplateParams(TypedDict, total=False):
    template: Literal["crescendo", "diminuendo", "swell", "tremolo", "filter_sweep", "pedal_sustain"]
    controller: int                  # CC number (default depends on template)
    start_value: int                 # Starting CC value (0-127)
    end_value: int                   # Ending CC value (0-127)
    peak_value: int                  # Peak for swell template
    peak_position: float             # 0.0-1.0 position of peak in bar
    lfo_rate: float                  # Hz for tremolo
    lfo_depth: int                   # Amplitude for tremolo
    apply_to_bars: Optional[List[int]]  # 1-based bar indices; None = all
    intensity: float                 # 0.0-1.0 master scale


def apply_cc_template(
    sml_clip: Dict[str, Any],
    params: CCTemplateParams,
) -> Dict[str, Any]:
    """Apply a CC template to an SML clip.
    
    Contract:
    - Input: SML clip dict, template parameters
    - Output: SML clip with bar-level `expression.cc` curves added/modified
    
    CC curve format in output:
        bar["expression"]["cc"] = {
            1: [{"time": 0.0, "value": 20}, {"time": 2.0, "value": 90}, ...],
            74: [...]
        }
    
    Common CC numbers:
    - CC1: Mod wheel
    - CC7: Volume
    - CC11: Expression
    - CC64: Sustain pedal
    - CC74: Filter cutoff (brightness)
    """
    pass

In [ ]:
# OpenAI Function Schema for Expression Agent

APPLY_CC_TEMPLATE_SCHEMA = {
    "name": "apply_cc_template",
    "description": "Apply a CC automation template to add expression to a clip.",
    "parameters": {
        "type": "object",
        "properties": {
            "template": {
                "type": "string",
                "enum": ["crescendo", "diminuendo", "swell", "tremolo", "filter_sweep", "pedal_sustain"],
                "description": "The type of CC curve to apply"
            },
            "controller": {
                "type": "integer",
                "minimum": 0,
                "maximum": 127,
                "description": "MIDI CC number (1=mod wheel, 11=expression, 64=pedal, 74=filter)"
            },
            "start_value": {"type": "integer", "minimum": 0, "maximum": 127},
            "end_value": {"type": "integer", "minimum": 0, "maximum": 127},
            "intensity": {"type": "number", "minimum": 0.0, "maximum": 1.0},
            "apply_to_bars": {
                "type": "array",
                "items": {"type": "integer", "minimum": 1},
                "description": "1-based bar indices to apply to; omit for all bars"
            }
        },
        "required": ["template"]
    }
}

## 3.3 apply_pitch_bend

**Purpose**: Add pitch bend curves for glides, scoops, and falls.

**Strategies**:
- `glide_into_next` - Bend from current note pitch toward next note
- `scoop` - Start below pitch, bend up (blues/jazz)
- `fall` - End of note bends down
- `vibrato` - Periodic pitch oscillation

In [ ]:
class PitchBendParams(TypedDict, total=False):
    strategy: Literal["glide_into_next", "scoop", "fall", "vibrato"]
    
    # For glide_into_next
    glide_start: float               # 0.0-1.0 position in note to start bend
    glide_curve: Literal["linear", "exponential", "s_curve"]
    
    # For scoop
    scoop_semitones: float           # How far below to start (e.g., 0.5, 1.0, 2.0)
    scoop_duration: float            # 0.0-1.0 portion of note for scoop
    
    # For fall
    fall_semitones: float            # How far to fall
    fall_start: float                # 0.0-1.0 position to start falling
    
    # For vibrato
    vibrato_rate: float              # Hz
    vibrato_depth: float             # Semitones (e.g., 0.25)
    vibrato_delay: float             # 0.0-1.0 delay before vibrato starts
    
    # Common
    pitch_bend_range: int            # Semitones (default: 2, synth-dependent)
    apply_to_notes: Optional[List[int]]  # 0-based note indices; None = all
    apply_to_bars: Optional[List[int]]   # 1-based bar indices; None = all


def apply_pitch_bend(
    sml_clip: Dict[str, Any],
    params: PitchBendParams,
) -> Dict[str, Any]:
    """Apply pitch bend curves to an SML clip.
    
    Contract:
    - Input: SML clip dict, pitch bend parameters
    - Output: SML clip with bar-level `expression.pitch_bend_curve` added
    
    Pitch bend values are 14-bit (0-16383), center at 8192.
    Default pitch bend range is 2 semitones, so:
    - 8192 = center (no bend)
    - 16383 = +2 semitones
    - 0 = -2 semitones
    
    Example output:
        bar["expression"]["pitch_bend_curve"] = [
            {"time": 0.0, "value": 8192},
            {"time": 0.75, "value": 11000},
            {"time": 1.0, "value": 8192}
        ]
    """
    pass

---
# 4. TRANSFORMATION TOOLS
---

## 4.1 transpose_clip

**Purpose**: Transpose all notes by a given interval.

In [ ]:
class TransposeParams(TypedDict, total=False):
    semitones: int                   # Interval to transpose (+/- semitones)
    octaves: int                     # Shorthand for semitones (1 octave = 12 semitones)
    clamp_to_range: bool             # Clamp to MIDI 0-127 (default: True)
    preserve_key: bool               # Transpose within key (diatonic) vs chromatic
    target_key: str                  # If preserve_key, transpose to this key


def transpose_clip(
    sml_clip: Dict[str, Any],
    params: TransposeParams,
) -> Dict[str, Any]:
    """Transpose all notes in an SML clip.
    
    Contract:
    - Input: SML clip, transpose parameters
    - Output: SML clip with transposed note pitches
    - Rests are unchanged
    - Notes clamped to MIDI range [0, 127] if clamp_to_range=True
    """
    pass

## 4.2 quantize_clip

**Purpose**: Snap note start times and durations to a grid.

In [ ]:
class QuantizeParams(TypedDict, total=False):
    grid: Literal["quarter", "eighth", "sixteenth", "triplet_eighth", "triplet_sixteenth"]
    strength: float                  # 0.0-1.0; 1.0 = full snap, 0.5 = halfway
    quantize_starts: bool            # Quantize note start times (default: True)
    quantize_durations: bool         # Quantize note durations (default: True)
    humanize_after: bool             # Apply small random offset after quantize


def quantize_clip(
    sml_clip: Dict[str, Any],
    params: QuantizeParams,
) -> Dict[str, Any]:
    """Quantize note timing to a grid.
    
    Contract:
    - Input: SML clip, quantize parameters
    - Output: SML clip with quantized timing
    - Strength < 1.0 moves notes partially toward grid
    """
    pass

## 4.3 apply_swing

**Purpose**: Apply swing feel by delaying off-beat notes.

In [ ]:
class SwingParams(TypedDict, total=False):
    amount: float                    # 0.0-1.0; 0.5 = straight, 0.67 = triplet swing
    grid: Literal["eighth", "sixteenth"]  # Which subdivision to swing
    velocity_accent: bool            # Also accent downbeats (default: False)


def apply_swing(
    sml_clip: Dict[str, Any],
    params: SwingParams,
) -> Dict[str, Any]:
    """Apply swing timing to a clip.
    
    Contract:
    - Input: SML clip, swing parameters
    - Output: SML clip with swung timing
    
    Swing delays every other subdivision:
    - amount=0.5: straight (no swing)
    - amount=0.67: triplet feel
    - amount=0.75: heavy shuffle
    """
    pass

## 4.4 invert_clip / retrograde_clip

**Purpose**: Classic compositional transformations.

In [ ]:
class InvertParams(TypedDict, total=False):
    axis_pitch: int                  # MIDI pitch to invert around (default: first note)
    preserve_rhythm: bool            # Keep original timing (default: True)


def invert_clip(
    sml_clip: Dict[str, Any],
    params: Optional[InvertParams] = None,
) -> Dict[str, Any]:
    """Invert pitches around an axis.
    
    For each note: new_pitch = 2 * axis_pitch - old_pitch
    """
    pass


class RetrogradeParams(TypedDict, total=False):
    preserve_bar_structure: bool     # Keep bars in order, reverse within (default: True)
    reverse_bars_too: bool           # Also reverse bar order (default: False)


def retrograde_clip(
    sml_clip: Dict[str, Any],
    params: Optional[RetrogradeParams] = None,
) -> Dict[str, Any]:
    """Reverse the order of notes (retrograde).
    
    By default, reverses notes within each bar but keeps bar order.
    """
    pass

---
# 5. HARMONY TOOLS
---

## 5.1 generate_chord_track

**Purpose**: Generate chord accompaniment from a melody or chord symbols.

In [ ]:
class ChordTrackParams(TypedDict, total=False):
    source: Literal["from_melody", "from_symbols"]  # How to derive chords
    chord_symbols: List[str]         # e.g., ["Cmaj7", "Dm7", "G7", "Cmaj7"]
    voicing: Literal["close", "open", "drop2", "drop3", "shell"]
    rhythm: Literal["whole", "half", "quarter", "comping"]  # Chord rhythm pattern
    register: Literal["low", "mid", "high"]  # Octave placement
    key: str                         # Key for analysis
    mode: str                        # "major" or "minor"


def generate_chord_track(
    melody_clip: Optional[Dict[str, Any]],
    params: ChordTrackParams,
) -> Dict[str, Any]:
    """Generate a chord track.
    
    Contract:
    - Input: Optional melody clip (for from_melody), chord parameters
    - Output: New SML clip with chord voicings
    
    If source="from_melody", analyzes melody to suggest chords.
    If source="from_symbols", uses provided chord_symbols list.
    """
    pass

## 5.2 generate_bassline

**Purpose**: Generate bass line from chord progression.

In [ ]:
class BasslineParams(TypedDict, total=False):
    style: Literal["root_only", "root_fifth", "walking", "arpeggiated", "syncopated"]
    chord_symbols: List[str]         # Chord progression
    octave: int                      # Base octave (default: 2)
    rhythm: Literal["whole", "half", "quarter", "eighth"]
    key: str
    mode: str


def generate_bassline(
    chord_clip: Optional[Dict[str, Any]],
    params: BasslineParams,
) -> Dict[str, Any]:
    """Generate a bass line from chords.
    
    Contract:
    - Input: Optional chord clip, bass parameters
    - Output: New SML clip with bass line
    """
    pass

## 5.3 arpeggiate_chords

**Purpose**: Convert block chords to arpeggios.

In [ ]:
class ArpeggiateParams(TypedDict, total=False):
    pattern: Literal["up", "down", "up_down", "down_up", "random", "as_played"]
    rate: Literal["eighth", "sixteenth", "triplet"]
    octave_range: int                # How many octaves to span (default: 1)
    gate: float                      # 0.0-1.0 note length relative to rate


def arpeggiate_chords(
    chord_clip: Dict[str, Any],
    params: ArpeggiateParams,
) -> Dict[str, Any]:
    """Convert block chords to arpeggios.
    
    Contract:
    - Input: SML clip with chords (multiple simultaneous notes)
    - Output: SML clip with arpeggiated notes
    """
    pass

---
# 6. PLAYBACK TOOLS
---

## 6.1 preview_clip

**Purpose**: Play an SML clip through FluidSynth for immediate preview.

**Wraps**: `OSCFacade.play_clip_from_sml`

In [ ]:
class PreviewParams(TypedDict, total=False):
    sf2_path: str                    # Path to SoundFont file
    bpm: int                         # Tempo (default: 120)
    loop: bool                       # Loop playback (default: False)
    program: int                     # MIDI program/patch number (0-127)
    channel: int                     # MIDI channel (0-15)


async def preview_clip(
    sml_clip: Dict[str, Any],
    params: Optional[PreviewParams] = None,
) -> bool:
    """Play an SML clip for preview.
    
    Contract:
    - Input: SML clip, playback parameters
    - Output: True if playback started successfully
    - Side effect: Audio plays through FluidSynth
    
    This is a thin wrapper around OSCFacade.play_clip_from_sml.
    """
    pass

---
# 7. STORAGE TOOLS
---

## 7.1 store_clip

**Purpose**: Store an SML clip to the database with tags.

**Wraps**: `OSCFacade.sml_to_dsl_clip` + `ClipService.create_clip_from_dsl`

In [ ]:
class StoreClipParams(TypedDict, total=False):
    tags: List[str]                  # Tags to attach
    auto_tag: bool                   # Auto-generate tags from content (default: True)
    name_override: str               # Override clip name


async def store_clip(
    sml_clip: Dict[str, Any],
    params: Optional[StoreClipParams] = None,
) -> int:
    """Store an SML clip to the database.
    
    Contract:
    - Input: SML clip, storage parameters
    - Output: clip_id of stored clip
    
    Auto-tagging derives tags from:
    - Key/scale (if detectable)
    - Tempo range
    - Note density
    - Instrument (if specified)
    """
    pass

## 7.2 search_clips

**Purpose**: Search for clips by tags or name pattern.

**Wraps**: `OSCFacade.search_clips`

In [ ]:
class SearchClipsParams(TypedDict, total=False):
    tags: List[str]                  # Tags to match (OR logic)
    name_pattern: str                # SQL LIKE pattern (e.g., "%bass%")
    limit: int                       # Max results (default: 10)


async def search_clips(
    params: SearchClipsParams,
) -> List[Dict[str, Any]]:
    """Search for clips in the database.
    
    Contract:
    - Input: Search parameters
    - Output: List of DSL clip dicts
    """
    pass

---
# 8. EXPORT TOOLS
---

## 8.1 export_to_midi

**Purpose**: Export clip or composition to MIDI file.

**Wraps**: `OSCFacade.dsl_to_midi_file`

In [ ]:
class ExportMidiParams(TypedDict, total=False):
    output_path: str                 # File path for .mid file
    include_all_tracks: bool         # Include all tracks (default: True)
    tempo_bpm: int                   # Override tempo
    include_cc: bool                 # Include CC automation (default: True)
    include_pitch_bend: bool         # Include pitch bend (default: True)


async def export_to_midi(
    clip_or_composition: Dict[str, Any],
    params: ExportMidiParams,
) -> str:
    """Export to MIDI file.
    
    Contract:
    - Input: SML/DSL clip or composition, export parameters
    - Output: Path to written MIDI file
    """
    pass

---
# 9. QA TOOLS
---

## 9.1 lint_clip

**Purpose**: Check clip for common issues and anti-patterns.

In [ ]:
class LintResult(TypedDict):
    issues: List[str]                # List of issue descriptions
    score: float                     # 0.0-1.0 quality score
    suggestions: List[str]           # Improvement suggestions


def lint_clip(sml_clip: Dict[str, Any]) -> LintResult:
    """Lint an SML clip for quality issues.
    
    Checks:
    - CC spam (too frequent CC changes)
    - Velocity extremes (all max or all min)
    - Empty bars
    - Pitch range issues (too wide or too narrow)
    - Duration consistency
    - Pitch bend returning to center
    """
    pass

## 9.2 summarize_clip

**Purpose**: Generate a human-readable summary of clip contents.

In [ ]:
class ClipSummary(TypedDict):
    name: str
    num_bars: int
    num_notes: int
    pitch_range: str                 # e.g., "C3-G5"
    detected_key: Optional[str]      # e.g., "C major"
    velocity_range: str              # e.g., "60-100"
    has_cc: bool
    has_pitch_bend: bool
    density: str                     # e.g., "sparse", "moderate", "dense"
    description: str                 # Natural language summary


def summarize_clip(sml_clip: Dict[str, Any]) -> ClipSummary:
    """Generate a summary of an SML clip.
    
    Useful for:
    - Displaying clip info in UI
    - Auto-generating tags
    - Explaining clip to user
    """
    pass

---
# GRAPH WIRING EXAMPLE
---

Example of wiring multiple nodes into a complete workflow.

In [ ]:
from langgraph.graph import StateGraph, END

# Complete state for the composition workflow
class ComposerState(TypedDict, total=False):
    # Input
    prompt: str
    
    # Generation params
    generate_params: GenerateClipParams
    humanize_params: dict  # HumanizeParams from velocity_humanizer
    cc_params: CCTemplateParams
    
    # Intermediate
    sml_clip: Dict[str, Any]
    validation_result: ValidationResult
    
    # Output
    clip_id: int
    export_path: str
    
    # Control
    error: str
    feedback: str


# Build the graph
def build_composer_graph():
    graph = StateGraph(ComposerState)
    
    # Add nodes
    graph.add_node("generate", generate_clip_node)
    graph.add_node("validate", validate_clip_node)
    graph.add_node("humanize", apply_velocity_humanization)  # from velocity_humanizer.ipynb
    graph.add_node("add_expression", apply_cc_template_node)
    graph.add_node("preview", preview_clip_node)
    graph.add_node("store", store_clip_node)
    graph.add_node("export", export_midi_node)
    
    # Wire edges
    graph.set_entry_point("generate")
    graph.add_edge("generate", "validate")
    graph.add_edge("validate", "humanize")
    graph.add_edge("humanize", "add_expression")
    graph.add_edge("add_expression", "preview")
    graph.add_edge("preview", "store")
    graph.add_edge("store", "export")
    graph.add_edge("export", END)
    
    return graph.compile()


# Usage
async def main():
    graph = build_composer_graph()
    
    state: ComposerState = {
        "prompt": "Create a jazzy piano riff in F major, 4 bars",
        "humanize_params": {
            "strategy": "meter_accent",
            "meter": "4/4",
            "intensity": 0.6,
        },
        "cc_params": {
            "template": "swell",
            "controller": 1,
            "intensity": 0.4,
        },
    }
    
    result = await graph.ainvoke(state)
    
    if result.get("error"):
        print(f"Error: {result['error']}")
    else:
        print(f"Stored clip ID: {result['clip_id']}")
        print(f"Exported to: {result['export_path']}")

---
# IMPLEMENTATION PRIORITY
---

Suggested order for implementing these tools:

## Phase 1: Core Pipeline (MVP)
1. `generate_clip_from_nl` - Already exists in clip_graph.py
2. `validate_sml_clip` - Essential for catching errors early
3. `humanize_velocities` - Designed in velocity_humanizer.ipynb
4. `preview_clip` - Already exists via OSCFacade
5. `store_clip` - Already exists via OSCFacade

## Phase 2: Expression
6. `apply_cc_template` - Add dynamics and expression
7. `apply_pitch_bend` - Glides and articulations

## Phase 3: Transformation
8. `transpose_clip` - Basic pitch manipulation
9. `quantize_clip` - Timing cleanup
10. `apply_swing` - Feel adjustment

## Phase 4: Harmony
11. `generate_chord_track` - Accompaniment
12. `generate_bassline` - Bass lines
13. `arpeggiate_chords` - Texture variation

## Phase 5: QA & Polish
14. `lint_clip` - Quality checks
15. `summarize_clip` - User-facing info
16. `invert_clip`, `retrograde_clip` - Advanced transforms